In [2]:
import numpy
from keras import backend as K
from keras.datasets import mnist
from keras.utils import np_utils

C:\Users\Suryaman\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from keras.layers import Dense, Dropout,Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D

In [4]:
from keras.models import Sequential
import pandas as pd

model = Sequential()


In [5]:
K.set_image_data_format('channels_last')
numpy.random.seed(0)

def load_data(path='input/mnist/mnist.npz'):
    """Loads the MNIST dataset.
    
    # Returns
        Tuple of Numpy arrays: `(x_train, y_train), (x_test, y_test)`.
    """
#     path = get_file(path, origin='https://s3.amazonaws.com/img-datasets/mnist.np/8z')
    f = numpy.load('input/mnist/mnist.npz')
    x_train = f['x_train']
    y_train = f['y_train']
    x_test = f['x_test']
    y_test = f['y_test']
    f.close()
    return (x_train, y_train), (x_test, y_test)

In [6]:
(X_train, y_train), (X_test, y_test)  = load_data()
training_set = pd.read_csv('input/train.csv')

test_set = pd.read_csv('input/test.csv')

In [7]:
X_train = X_train / 255
X_test2 = X_test / 255

In [8]:
import matplotlib.pyplot as plt

print("the number of training examples = %i" % X_train.shape[0])
print("the number of classes = %i" % len(numpy.unique(y_train)))
print("Dimention of images = {:d} x {:d}  ".format(X_train[1].shape[0],X_train[1].shape[1])  )

#This line will allow us to know the number of occurrences of each specific class in the data
unique, count= numpy.unique(y_train, return_counts=True)
print("The number of occuranc of each class in the dataset = %s " % dict (zip(unique, count) ), "\n" )
 
images_and_labels = list(zip(X_train,  y_train))
for index, (image, label) in enumerate(images_and_labels[:12]):
    plt.subplot(5, 4, index + 1)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('label: %i' % label)

the number of training examples = 60000
the number of classes = 10
Dimention of images = 28 x 28  
The number of occuranc of each class in the dataset = {0: 5923, 1: 6742, 2: 5958, 3: 6131, 4: 5842, 5: 5421, 6: 5918, 7: 6265, 8: 5851, 9: 5949}  



In [9]:
X_train = X_train.reshape(X_train.shape[0], 28, 28 , 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 28, 28 , 1).astype('float32')
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [10]:
from keras.layers import Dropout

model.add(Conv2D(40, kernel_size=5, padding="same",input_shape=(28, 28, 1), activation = 'relu'))
model.add(Conv2D(50, kernel_size=5, padding="valid", activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.2))

In [11]:
model.add(Conv2D(70, kernel_size=3, padding="same", activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(100, kernel_size=3, padding="valid", activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

In [12]:
from keras.layers.core import Activation

model.add(Flatten())
model.add(Dense(100))
model.add(Activation("relu"))
model.add(Dropout(0.2))

model.add(Dense(10))
model.add(Activation("softmax"))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs= 32 , batch_size=200, validation_split = 0.2)
scores = model.evaluate(X_test, y_test, verbose = 10 )
print ( scores )

Train on 48000 samples, validate on 12000 samples
Epoch 1/32
48000/48000 [==============================] - 1656s 35ms/step - loss: 0.3683 - acc: 0.8797 - val_loss: 0.0759 - val_acc: 0.9763
Epoch 2/32
48000/48000 [==============================] - 1649s 34ms/step - loss: 0.0876 - acc: 0.9739 - val_loss: 0.0492 - val_acc: 0.9848
Epoch 3/32
48000/48000 [==============================] - 1642s 34ms/step - loss: 0.0575 - acc: 0.9829 - val_loss: 0.0391 - val_acc: 0.9880
Epoch 4/32
48000/48000 [==============================] - 1656s 35ms/step - loss: 0.0453 - acc: 0.9867 - val_loss: 0.0425 - val_acc: 0.9873
Epoch 5/32
48000/48000 [==============================] - 1650s 34ms/step - loss: 0.0389 - acc: 0.9882 - val_loss: 0.0381 - val_acc: 0.9874
Epoch 6/32
48000/48000 [==============================] - 1651s 34ms/step - loss: 0.0310 - acc: 0.9903 - val_loss: 0.0321 - val_acc: 0.9902
Epoch 7/32
48000/48000 [==============================] - 1662s 35ms/step - loss: 0.0299 - acc: 0.9906 - val_l

In [14]:
test_set = (test_set.values).reshape(test_set.shape[0], 28, 28 , 1).astype('float32')

res = model.predict(test_set)
res = numpy.argmax(res,axis = 1)
res = pd.Series(res, name="Label")
submission = pd.concat([pd.Series(range(1 ,28001) ,name = "ImageId"),   res],axis = 1)
submission.to_csv("cnn_mnist_datagen.csv",index=False)
submission.head(10)

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
5,6,7
6,7,0
7,8,3
8,9,0
9,10,3
